<a href="https://colab.research.google.com/github/Rayckey/3DHelper/blob/master/Lab_2_DDE_oscillations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Lab 2: Oscillations in Delay Differential Equations**

Along with sensitive negative feedback, time delays are a critical ingredient
in causing oscillations in dynamical systems. Without any delays, even
the most sensitive feedback cannot produce oscillations in a system of
differential equations of the kind we study in this class.

Delays in a model can be either implicit (arising from the structure of
the model, particularly the number of variables) or explicit. Explicit
delays occur when a term in a differential equation is a function of the
value of the state variable some time ago. For example, we might have
$X'(t)=2X(t-5)$, where $X(t-5)$ is the value of $X$ 5 time units before
the present time. Such equations, which explicitly include time delays,
are called **delay differential equations**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulating Delay Differential Equations

In order to simulate a delay differential equation, we have to find the
value of $x$ $\tau$ time units ago, plug this value into the equation,
and then integrate one time step, just as we did with Euler’s method.
The `ddeint` library (given below because the version available online
doesn't work with current versions of Numpy and Scipy) carries out this process.

<font color='green'><u><b> Exercise 1.</b></u> </font> Evaluate the cell below. Nothing visible will happen when you do.

In [ ]:
# @title Run `ddeint` (simple Differential Delay Equation solver built on top of Scipy's odeint) {display-mode:'form'}
"""
This module implements ddeint, a simple Differential Delay Equation
solver built on top of Scipy's odeint """

# REQUIRES Numpy and Scipy.
import numpy as np
import scipy.integrate
import scipy.interpolate


class ddeVar:
    """
    The instances of this class are special function-like
    variables which store their past values in an interpolator and
    can be called for any past time: Y(t), Y(t-d).
    Very convenient for the integration of DDEs.
    """

    def __init__(self, g, tc=0):
        """ g(t) = expression of Y(t) for t<tc """

        self.g = g
        self.tc = tc
        # We must fill the interpolator with 2 points minimum

        self.interpolator = scipy.interpolate.interp1d(
            np.array([tc - 1, tc]),  # X
            np.array([self.g(tc), self.g(tc)]).T,  # Y
            kind="linear",
            bounds_error=False,
            fill_value=self.g(tc)
        )

    def update(self, t, Y):
        """ Add one new (ti,yi) to the interpolator """
        Y2 = Y if (Y.size == 1) else np.array([Y]).T
        self.interpolator = scipy.interpolate.interp1d(
            np.hstack([self.interpolator.x, [t]]),  # X
            np.hstack([self.interpolator.y, Y2]),  # Y
            kind="linear",
            bounds_error=False,
            fill_value=Y
        )

    def __call__(self, t=0):
        """ Y(t) will return the instance's value at time t """

        return self.g(t) if (t <= self.tc) else self.interpolator(t)


class dde(scipy.integrate.ode):
    """
    This class overwrites a few functions of ``scipy.integrate.ode``
    to allow for updates of the pseudo-variable Y between each
    integration step.
    """

    def __init__(self, f, jac=None):
        def f2(t, y, args):
            return f(self.Y, t, *args)

        scipy.integrate.ode.__init__(self, f2, jac)
        self.set_f_params(None)

    def integrate(self, t, step=0, relax=0):

        scipy.integrate.ode.integrate(self, t, step, relax)
        self.Y.update(self.t, self.y)
        return self.y

    def set_initial_value(self, Y):

        self.Y = Y  #!!! Y will be modified during integration
        scipy.integrate.ode.set_initial_value(self, Y(Y.tc), Y.tc)

def ddeint(func, g, tt, fargs=None):
    """ Solves Delay Differential Equations

    Similar to scipy.integrate.odeint. Solves a Delay differential
    Equation system (DDE) defined by

        Y(t) = g(t) for t<0
        Y'(t) = func(Y,t) for t>= 0

    Where func can involve past values of Y, like Y(t-d).
ValueError: setting an array element with a sequence
      systems).

    tt
      The vector of times [t0, t1, ...] at which the system must
      be solved.
ValueError: setting an array element with a sequence
    fargs
      Additional arguments to be passed to parameter ``func``, if any.


    Examples
    ---------

    We will solve the delayed Lotka-Volterra system defined as

        For t < 0:
        x(t) = 1+t
        y(t) = 2-t

        For t >= 0:
        dx/dt =  0.5* ( 1- y(t-d) )
        dy/dt = -0.5* ( 1- x(t-d) )

    The delay ``d`` is a tunable parameter of the model.

    >>> import numpy as np
    >>> from ddeint import ddeint
    >>>
    >>> def model(XY,t,d):
    >>>     x, y = XY(t)
    >>>     xd, yd = XY(t-d)
    >>>     return np.array([0.5*x*(1-yd), -0.5*y*(1-xd)])
    >>>
    >>> g = lambda t : np.array([1+t,2-t]) # 'history' at t<0
    >>> tt = np.linspace(0,30,20000) # times for integration
    >>> d = 0.5 # set parameter d ")
    >>> yy = ddeint(model,g,tt,fargs=(d,)) # solve the DDE !

    """

    dde_ = dde(func)
    dde_.set_initial_value(ddeVar(g, tt[0]))
    dde_.set_f_params(fargs if fargs else [])
    results = [dde_.integrate(dde_.t + dt) for dt in np.diff(tt)]

    #Correct array orientation to match modern syntax and inserts
    #initial value at beginning
    results = np.array([results[i][0] for i in range(len(results))])
    results = np.insert(results, 0, g(tt[0]))

    return results

The syntax for the `ddeint` function is `sol=ddeint(DDE, history, times)`, where `history`is a function describing the state variable values before the simulation began, and `times` is a list of steps for simulation, like you made in previous labs. For example, here’s a simulation of a version of the logistic model with a time delay and an oscillating history.

```
import numpy as np
import matplotlib.pyplot as plt

#Define time delay
tau = 8

#Define differential equation(s) to simulate
def DDE(X,t):
  Xprime = 0.1*X(t)*(1-X(t-tau)/100)
  return Xprime

#Define history function
def history(t):
  X = 5*np.cos(t)
  return X

#Make a list of times at which to output values and run simulation
times = np.linspace(0, 400, 4000)
solution = ddeint(DDE, history, times)

#Plot time series
plt.plot(times, solution)
plt.xlabel("Time")
plt.ylabel("Populations")
plt.show()
```


# A Model of Respiration

This lab will focus on the Mackey-Glass model of respiration described in your text and in lecture. (Similar equations are used to model delays in other physiological processes.) The purpose of this model is to study respiration rate, which can be altered in certain diseases like heart failure or stroke. However, breathing is mostly controlled by $\textrm{CO}_{2}$ content in the blood, so we'll actually model $\textrm{CO}_{2}$.

The exact state variable in this model is the concentration of carbon dioxide
in arterial blood _at the lungs_, which we’ll call $X$. The fact that the blood is at the lungs will soon become important.

We'll make the simplifying assumption that the body’s rate of production of $\textrm{CO}_{2}$ is a constant, which we’ll call $L$.

Excretion is a little more complicated. The total $\textrm{CO}_{2}$ exhalation rate is ($\textrm{CO}_{2}$ / breath) $×$ (breaths / minute). The ($\textrm{CO}_{2}$ / breath) term is proportional to the amount of $\textrm{CO}_{2}$ in the blood, which is just $X$. For simplicity, we'll make the proportionality constant be 1.

A person's breathing rate is a sigmoidal function of its concentration in the blood. The function $f(X)=\frac{V_{max}X^{n}}{h+X^{n}}$ does the job for sufficiently large values of $n$. Thus, the overall $\textrm{CO}_{2}$ exhalation rate is $\frac{V_{max}X^{n}}{h+X^{n}} X$.

There is one complication in this model – a delay between gas exchange
in the lungs and the effect on $\textrm{CO}_{2}$-monitoring neurons in
the brain. In simple terms, it takes time for blood to get from the
lungs to the brain. Therefore, the brain is responding not to the
current $\textrm{CO}_{2}$ concentration in the blood but to the
concentration some time ago. (In the body, this delay is on the order of
0.2 minutes.) Thus, the $\textrm{CO}_{2}$ excretion function really
needs to be $f(X)=\frac{V_{max}X(t-\tau)^{n}}{h+X(t-\tau)^{n}} X(t)$.

Putting everything together, our model is $X'(t)=L-\frac{V_{max}X(t-\tau)^{n}}{h+X(t-\tau)^{n}}x(t)$. The parameter values we will use are $L=6$, $V_{max}=20$, $n=5$, $h=400$ and $\tau=0.2$. With these parameters, our time units are minutes.

<font color='green'><u><b> Exercise 2.</b></u> </font> Following the example above, simulate the Mackey-Glass model for 30 minutes. Assign the output to a variable and plot your results. For the history, use a function that returns 0.

<font color='green'><u><b> Exercise 3.</b></u></font> The Mackey-Glass model simulates CO<sub>2</sub> concentration, but we are interested in breathing rate. The two are linked by the function $v(X)=\frac{V_{max}X^{n}}{h+X^{n}}$, the same function used in actually formulating the model. Use this function to
convert your simulation results into breathing rate. (When using Numpy, you can just define a function representing the formula and apply it directly to your desired input.)

<font color='green'><u><b> Exercise 4.</b></u></font> Plot your results from the previous exercise. Make sure the time axis is appropriately scaled.

<font color='green'><u><b> Exercise 5.</b></u></font> Briefly describe the behavior you see in your plots.

In [ ]:
#write text here

# Drivers of Oscillation in the Mackey-Glass Model

We have said that a delay is necessary to cause persistent oscillations in the
Mackey-Glass model, but how much of a delay? Is the mere presence of any
delay sufficient or is there a threshold?

<font color='green'><u><b> Exercise 6. </b></u></font> Simulate the model for at least three different values of $\tau$. Since all we’re interested in is the presence or absence of oscillations, you don’t need to convert the $X$ values to breathing rate.

<font color='green'><u><b> Exercise 7. </b></u></font> By manipulating $\tau$, approximate the minimum delay needed for persistent oscillations.

The other key ingredient in oscillations is a sensitive negative
feedback loop. Again, we are interested in the question of how much
sensitivity is necessary to produce persistent oscillations.

<font color='green'><u><b> Exercise 8. </b></u></font> “Sensitivity” in the Mackey-Glass model is embodied in the steepness of the sigmoid function controlling respiration. Which parameter controls this steepness? If you’re not sure, plot the function and manipulate parameters to check.

In [ ]:
#write text and/or code here

<font color='green'><u><b> Exercise 9. </b></u></font> Copy your code from Exercise 7 and set $\tau$ to the minimum value you found to generate oscillations. Manipulate your sensitivity parameter to find the minimum sensitivity at which persistent oscillations appear.

Manipulating parameters together can have very different effects from
manipulating them separately.

<font color='green'><u><b> Exercise 10. </b></u></font> Pick at least five values of $\tau$ and, for each one, approximate the minimum sensitivity necessary for persistent oscillations. Make lists of these values and plot one list against the other using `plt.scatter`.

<font color='green'><u><b> Exercise 11. </b></u></font> Describe the relationship you found in the previous exercise.

In [ ]:
#write text here